In [ ]:
# Imports
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from timeit import default_timer as timer

plt.rcParams["figure.figsize"] = (20,10)

In [ ]:
# Parameters 
# ATTEMPT_NAME="LSTM_BASELINE"
# DATASET_FILE_PATH= ["Datasets\\uniformdata.xlsx", "Datasets\\uniformdata.xlsx", "Datasets\\uniformdata.xlsx", "Datasets\\uniformdata.xlsx", "Datasets\\uniformdata.xlsx"]
# DATASET_SHEET_TITLE= ["data_uniforma", "data_uniformb", "data_uniformc", "data_uniformd", "data_uniforme"]
# GRANULARITY=10
# STEP_SIZE_SLIDING_WINDOW=5
# PAST_HISTORY=20
# FUTURE_TARGET=1
# VAL_PERCENT=0.1
# EPOCHS=5
# BATCH_SIZE=10
# SMOOTHING=50
# SHUFFLE_BUFFER_SIZE=100
# MEAN=False
# USE_REF_POINTS=False
# REF_POINT1=0
# REF_POINT2=7

In [ ]:
def run_all(DATASET_FILE_PATH, DATASET_SHEET_TITLE, GRANULARITY, STEP_SIZE_SLIDING_WINDOW, PAST_HISTORY, 
            FUTURE_TARGET, VAL_PERCENT, EPOCHS, BATCH_SIZE, SMOOTHING, ATTEMPT_NAME, SHUFFLE_BUFFER_SIZE, MEAN,
            USE_REF_POINTS, REF_POINT1, REF_POINT2): 
    batch_train, batch_val, batch_test, test_ground_truth, train_slices, val_slices, test_data_indexes, data_shape = process_data(
        DATASET_FILE_PATH, DATASET_SHEET_TITLE, GRANULARITY, SMOOTHING, VAL_PERCENT, PAST_HISTORY, 
        FUTURE_TARGET, STEP_SIZE_SLIDING_WINDOW, BATCH_SIZE, EPOCHS, SHUFFLE_BUFFER_SIZE, MEAN, 
        USE_REF_POINTS, REF_POINT1, REF_POINT2)
    model, training_history, training_time = run_cnn(data_shape, batch_train, batch_val, train_slices, 
                                                      val_slices, BATCH_SIZE, EPOCHS, FUTURE_TARGET) 
    evaluate_results(model, training_history, test_ground_truth, batch_test, test_data_indexes, training_time)

In [ ]:
# Load components
%run ./data_management_component.ipynb

In [ ]:
%run ./cnn_baseline.ipynb

In [ ]:
%run ./lstm_baseline.ipynb

In [ ]:
%run ./nn_baseline.ipynb

In [ ]:
%run ./parallel_model.ipynb

In [ ]:
%run ./parallel_model_deep.ipynb

In [ ]:
%run ./evaluation_component.ipynb

In [ ]:
DATASET_FILE_PATH=["Datasets\\SingleSessionData.xlsx","Datasets\\SingleSessionData.xlsx","Datasets\\SingleSessionData.xlsx","Datasets\\SingleSessionData.xlsx","Datasets\\SessionToSessionOne.xlsx","Datasets\\SessionToSessionOne.xlsx","Datasets\\SessionToSessionOne.xlsx","Datasets\\SessionToSessionOne.xlsx","Datasets\\SessionToSessionTwo.xlsx","Datasets\\SessionToSessionTwo.xlsx","Datasets\\SessionToSessionTwo.xlsx","Datasets\\SessionToSessionTwo.xlsx","Datasets\\SessionToSessionThree.xlsx","Datasets\\SessionToSessionThree.xlsx","Datasets\\SessionToSessionThree.xlsx","Datasets\\SessionToSessionThree.xlsx","Datasets\\SessionToSessionFour.xlsx","Datasets\\SessionToSessionFour.xlsx","Datasets\\SessionToSessionFour.xlsx","Datasets\\SessionToSessionFour.xlsx"]
DATASET_SHEET_TITLE=["data_Uniform","data_NonUniform","data_NonUniformWithPause","data_NonUniformTwo","data_Uniform","data_NonUniform","data_NonUniformWithPause","data_UniformWithPause","data_Uniform","data_UniformTwo","data_NonUniform","data_NonUniformTwo","data_Uniform","data_UniformTwo","data_NonUniform","data_NonUniformTwo","data_NonUniform","data_NonUniformTwo","data_Uniform","data_UniformTwo"]
GRANULARITY=10
STEP_SIZE_SLIDING_WINDOW=1
PAST_HISTORY=100
FUTURE_TARGET=1
VAL_PERCENT=0.15
EPOCHS=100 # Max epochs, in case early stopping doesn't take effect
MIN_DELTA=0.0001
PATIENCE=10
BATCH_SIZE=500
SMOOTHING=50
SHUFFLE_BUFFER_SIZE=0
MEAN=False
USE_REF_POINTS=False
REF_POINT1=0
REF_POINT2=7

plt.rcParams["figure.figsize"] = (20,10)

# Load data
batch_train, batch_val, batch_test, batch_plot, test_ground_truth, train_slices, val_slices, test_data_indexes, data_shape = process_data(
    DATASET_FILE_PATH, DATASET_SHEET_TITLE, GRANULARITY, SMOOTHING, VAL_PERCENT, PAST_HISTORY, 
    FUTURE_TARGET, STEP_SIZE_SLIDING_WINDOW, BATCH_SIZE, EPOCHS, SHUFFLE_BUFFER_SIZE, MEAN, 
    USE_REF_POINTS, REF_POINT1, REF_POINT2)

In [ ]:
ATTEMPT_NAME="CNN_BASELINE"
KERNEL_SIZE=10
FILTERS=32
PADDING="valid"

# TODO TOMORROW:
# Find den rigtige optimizer (rmsprop eller adagrad)
# Kod så vi tester på en hel session på en gang
# Prøv med shufflebuffer igen, det kan sgu godt være
# Prøv uden de dataset hvor sensoren er død

# Lav comparison mellem følgende lstm'er: rmsprop, adagrad, adam (done), shufflebuffer, earlystopping-restore-values
# TRY RESTORE PARAMETERS ON EARLY STOPPING!

model, training_history, training_time = run_cnn(data_shape, batch_train, batch_val, train_slices, val_slices, BATCH_SIZE, EPOCHS, FUTURE_TARGET, KERNEL_SIZE, FILTERS, PADDING, MIN_DELTA, PATIENCE) 
evaluate_results(model, training_history, test_ground_truth, batch_test, batch_plot, test_data_indexes, training_time)

In [ ]:
ATTEMPT_NAME="NN_BASELINE"
DENSE_SIZE = 200
DENSE_LAYER_NUM = 1

model, training_history, training_time = run_nn(data_shape, batch_train, batch_val, train_slices, val_slices, BATCH_SIZE, EPOCHS, FUTURE_TARGET, DENSE_SIZE, DENSE_LAYER_NUM, MIN_DELTA, PATIENCE)
evaluate_results(model, training_history, test_ground_truth, batch_test, test_data_indexes, training_time)

In [ ]:
ATTEMPT_NAME="LSTM_BASELINE"
LAYER_SIZE = 32

model, training_history, training_time = run_lstm(data_shape, batch_train, batch_val, train_slices, val_slices, BATCH_SIZE, EPOCHS, FUTURE_TARGET, MIN_DELTA, PATIENCE, LAYER_SIZE)
evaluate_results(model, training_history, test_ground_truth, batch_test, test_data_indexes, training_time)

In [ ]:
ATTEMPT_NAME="PARALLEL_BASELINE"
KERNEL_SIZE=10
FILTERS=32
PADDING="valid"
LAYER_SIZE = 32

model, training_history, training_time = run_parallel(data_shape, batch_train, batch_val, train_slices, val_slices, BATCH_SIZE, EPOCHS, FUTURE_TARGET, MIN_DELTA, PATIENCE, KERNEL_SIZE, FILTERS, PADDING, LAYER_SIZE)
evaluate_results(model, training_history, test_ground_truth, batch_test, test_data_indexes, training_time)

In [ ]:
ATTEMPT_NAME="PARALLEL_DEEP"
KERNEL_SIZE=10
FILTERS=32
PADDING="valid"
LAYER_SIZE = 32
DENSE_SIZE = 32

model, training_history, training_time = run_parallel_deep(data_shape, batch_train, batch_val, train_slices, val_slices, BATCH_SIZE, EPOCHS, FUTURE_TARGET, MIN_DELTA, PATIENCE, KERNEL_SIZE, FILTERS, PADDING, LAYER_SIZE, DENSE_SIZE)
evaluate_results(model, training_history, test_ground_truth, batch_test, test_data_indexes, training_time)